# Battle of Neighborhood

In [12]:
# Import required libraries
import pandas as pd
import numpy as np
import io
from bs4 import BeautifulSoup
import requests

## Get Toronto Data

In [51]:
# Get Data
wikipage = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(wikipage)

# Upon successful download parse the data into a BeautifulSoup object 
if result.status_code == 200:
    soup = BeautifulSoup(result.content, "html.parser")
    

table = soup.find('table',{'class':'wikitable sortable'})

# loop through all the rows and pull the text
new_table = []
for row in table.find_all('tr')[1:]:
    column_marker = 0
    columns = row.find_all('td')
    new_table.append([column.get_text().strip() for column in columns])

# Load data into a dataframe     
df = pd.DataFrame(new_table, columns=['Postcode', 'Borough', 'Neighborhood'])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [52]:
# Exclude Not Assigned Borough
df1=df[df['Borough']!='Not assigned']
df1.head()


#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto = df1.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))
find_na = df_toronto['Neighborhood'] == "Not assigned"
df_toronto.loc[find_na, 'Neighborhood'] = df_toronto.loc[find_na, 'Borough']
df_toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df_toronto.shape

(103, 3)

## Get Latitude and the longitude coordinates

In [16]:
# Download longitude Latitude data and load into a dataframe
url = "http://cocl.us/Geospatial_data"
lat_long= requests.get(url).text
df_lat_long = pd.read_csv(io.StringIO(lat_long))


# Merge Neighborhood and Longitude and Latitude data 
df_lat_long=df_lat_long.rename(columns={'Postal Code': 'Postcode'})
toronto_df=pd.merge(df_toronto, df_lat_long, on='Postcode')
toronto_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
print('The dataframe has {} boroughs and {} neighbohoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighbohoods.


# Cluster the neighborhoods in Toronto

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe


# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.



In [21]:
#Get longitude and Latitude of Toronto
t_address = "Toronto, ON"
t_geolocator = Nominatim(user_agent="toronto_explorer")
t_location = t_geolocator.geocode(t_address)
t_latitude = t_location.latitude
t_longitude = t_location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(t_latitude, t_longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [22]:
# create map of Toronto using latitude and longitude values and add markers
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# select only boroughs that contain the word Toronto 
toronto_df_only = toronto_df[toronto_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
# toronto_df_only = toronto_df

# add markers
for lat, lng, borough, neighborhood in zip(
        toronto_df_only['Latitude'], 
        toronto_df_only['Longitude'], 
        toronto_df_only['Borough'], 
        toronto_df_only['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

## Explore the neighborhood using Foursquare

In [23]:
CLIENT_ID = 'NJVB2OJDMAEKWUCMCKKTWP53T35DSXPTQAKDKXQ4ZFU21MV0' # Foursquare ID
CLIENT_SECRET = 'NPEIBN5HUYOLOFBRPKXXQBEFF5XT22O3EGVT0NV4O3G3L5KO' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

##  Slice the original dataframe and create a new dataframe of the Scarborough Neighborhood

In [24]:
scarborough_data = toronto_df[toronto_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Get the geographical coordinates of Scarborough

In [26]:
address = 'Scarborough, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [27]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Explore Neighborhoods

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [29]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 89 venues in 17 neighborhoods.


In [30]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [31]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [32]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

There are 80 distinct venues in 55 categories.


## Analyze Each Neighborhood

In [33]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood by taking mean of frequency of occurrence of each category

In [34]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,...,0.000000,0.000,0.000,0.000000,0.20,0.000000,0.0,0.000000,0.00,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.0,0.000000,0.00,0.0
2,Cedarbrae,0.000000,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.125000,0.00,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.083333,0.00,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,...,0.000000,0.125,0.125,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.25,0.0
9,"Malvern, Rouge",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0


### Each Neighborhood with top 5 venues

In [35]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2
5          Accessories Store   0.0
6                       Park   0.0
7              Jewelry Store   0.0
8          Korean Restaurant   0.0
9             Medical Center   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink  0.25
1      General Entertainment  0.25
2                       Café  0.25
3            College Stadium  0.25
4          Accessories Store  0.00
5                       Park  0.00
6              Jewelry Store  0.00
7          Korean Restaurant  0.00
8  Latin American Restaurant  0.00
9                     Lounge  0.00


----Cedarbrae----
                  venue  freq
0    Athletics & Sports  0.12
1   Fried Chicken Joint  0.12
2                Bakery  0.12
3                  Bank  0.12
4   

In [36]:
## Sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Display top ten venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Intersection,Bank,Italian Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,Bakery,American Restaurant,Grocery Store,General Entertainment,Gas Station
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Ice Cream Shop,Park,Bus Station,Metro Station,Intersection,Soccer Field,Fried Chicken Joint,Coffee Shop
7,"Guildwood, Morningside, West Hill",Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
8,"Kennedy Park, Ionview, East Birchmount Park",Train Station,Hobby Shop,Department Store,Coffee Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store


In [37]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                                        Bar
2nd Most Common Venue                 Construction & Landscaping
3rd Most Common Venue                      Vietnamese Restaurant
4th Most Common Venue                             Clothing Store
5th Most Common Venue                                        Gym
6th Most Common Venue                              Grocery Store
7th Most Common Venue                      General Entertainment
8th Most Common Venue                                Gas Station
9th Most Common Venue                        Fried Chicken Joint
10th Most Common Venue                      Fast Food Restaurant
Name: 11, dtype: object

## Cluster Scarborough Neighborhood using k-means

In [38]:
# Cluster neighborhood into three clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2], dtype=int32)

In [42]:
# Dataframe that includes the cluster as well as the top ten venues for each neighborhood
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Scarborough Neighborhood Cluster

In [44]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,0,Accessories Store,Auto Garage,Bakery,Sandwich Place,Middle Eastern Restaurant,Smoke Shop,General Entertainment,Grocery Store,Clothing Store,Fried Chicken Joint


In [45]:
scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint


In [46]:
scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,Scarborough,-79.160497,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,-79.188711,2,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
3,Scarborough,-79.216917,2,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,-79.239476,2,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
5,Scarborough,-79.239476,2,Smoke Shop,Jewelry Store,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,Scarborough,-79.262029,2,Train Station,Hobby Shop,Department Store,Coffee Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,-79.284577,2,Bakery,Bus Line,Ice Cream Shop,Park,Bus Station,Metro Station,Intersection,Soccer Field,Fried Chicken Joint,Coffee Shop
8,Scarborough,-79.239476,2,Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,-79.273304,2,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,Bakery,American Restaurant,Grocery Store,General Entertainment,Gas Station


# Explore New York City Neighborhoods

In [64]:
import json # library to handle JSON files
# Get Data
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json

# Load data into a dataframe
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']    

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Load data
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [68]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Get Latitude and the longitude coordinates

In [69]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [70]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Slice the original dataframe and create a new dataframe of Queens Neighborhood

In [75]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138
5,Queens,Corona,40.742382,-73.856825
6,Queens,Forest Hills,40.725264,-73.844475
7,Queens,Kew Gardens,40.705179,-73.829819
8,Queens,Richmond Hill,40.697947,-73.831833
9,Queens,Flushing,40.764454,-73.831773


In [83]:
queens_data.shape

(81, 4)

In [72]:
address = 'Queens, NY'
geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [73]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### Explore the Long Island City neighborhood in Queens

In [74]:
queens_data.loc[10, 'Neighborhood']

'Long Island City'

In [78]:
#Long Island City Latitude and Longitude values

latitudes = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
longitudes = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

names = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(names, 
                                                               latitudes, 
                                                               longitudes))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


### Top 100 venues in Long Island City neighborhood within a radius of 500 meters

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [84]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Found 2113 venues in 81 neighborhoods.


In [85]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
3,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [86]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,20,20,20,20,20,20
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,22,22,22,22,22,22
Bay Terrace,37,37,37,37,37,37
Bayside,74,74,74,74,74,74
Bayswater,1,1,1,1,1,1
Beechhurst,17,17,17,17,17,17
Bellaire,13,13,13,13,13,13


In [87]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

There are 1742 distinct venues in 278 categories.


In [88]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Astoria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood by taking mean of frequency of occurrence of each category

In [89]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.050000,0.0,0.000000,0.000000
1,Astoria,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.010000,0.0,0.000000,0.000000
2,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Auburndale,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,Bay Terrace,0.027027,0.000000,0.000000,0.054054,0.000000,0.000000,0.0,0.0,0.0,...,0.027027,0.000000,0.000000,0.0,0.027027,0.000000,0.000000,0.0,0.054054,0.000000
5,Bayside,0.000000,0.000000,0.000000,0.040541,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.013514,0.0,0.000000,0.013514,0.000000,0.0,0.000000,0.013514
6,Bayswater,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,Beechhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,Bellaire,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,Belle Harbor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


### Each Neighborhood with top 5 venues

In [90]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0       Surf Spot  0.20
1   Metro Station  0.10
2  Sandwich Place  0.10
3      Donut Shop  0.05
4     Pizza Place  0.05


----Astoria----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1                        Bar  0.06
2          Indian Restaurant  0.04
3           Greek Restaurant  0.04
4                 Hookah Bar  0.04


----Astoria Heights----
                venue  freq
0         Bus Station  0.08
1         Supermarket  0.08
2                Food  0.08
3  Chinese Restaurant  0.08
4       Bowling Alley  0.08


----Auburndale----
                venue  freq
0  Italian Restaurant  0.09
1                 Bar  0.05
2            Pharmacy  0.05
3           Pet Store  0.05
4          Comic Shop  0.05


----Bay Terrace----
            venue  freq
0  Clothing Store  0.11
1      Shoe Store  0.08
2      Kids Store  0.05
3  Cosmetics Shop  0.05
4      Donut Shop  0.05


----Bayside----
                venue  freq
0                

In [95]:
# Sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Top venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(48)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Playground,Restaurant,Bus Stop,Café,Board Shop,Beach,Thai Restaurant
1,Astoria,Middle Eastern Restaurant,Bar,Seafood Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Bakery,Mediterranean Restaurant,Food Truck,Café
2,Astoria Heights,Plaza,Bakery,Burger Joint,Food,Supermarket,Bowling Alley,Chinese Restaurant,Laundromat,Bus Station,Playground
3,Auburndale,Italian Restaurant,Comic Shop,Train,Supermarket,Furniture / Home Store,Mattress Store,Korean Restaurant,Bar,Fast Food Restaurant,Gymnastics Gym
4,Bay Terrace,Clothing Store,Shoe Store,Cosmetics Shop,American Restaurant,Donut Shop,Kids Store,Women's Store,Mobile Phone Shop,Furniture / Home Store,Gift Shop
5,Bayside,Bar,Pizza Place,Chinese Restaurant,Sushi Restaurant,American Restaurant,Indian Restaurant,Pub,Mexican Restaurant,Bakery,Italian Restaurant
6,Bayswater,Playground,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market
7,Beechhurst,Chinese Restaurant,Shopping Plaza,Boutique,Supermarket,Outdoors & Recreation,Shopping Mall,Park,Donut Shop,Optical Shop,Italian Restaurant
8,Bellaire,Deli / Bodega,Halal Restaurant,Moving Target,Breakfast Spot,Chinese Restaurant,Greek Restaurant,Grocery Store,Bus Station,Coffee Shop,Gym
9,Belle Harbor,Beach,Pub,Spa,Deli / Bodega,Donut Shop,Italian Restaurant,Boutique,Chinese Restaurant,Pharmacy,Mexican Restaurant


In [93]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                  Long Island City
1st Most Common Venue              Coffee Shop
2nd Most Common Venue                    Hotel
3rd Most Common Venue              Pizza Place
4th Most Common Venue                      Bar
5th Most Common Venue                     Café
6th Most Common Venue       Mexican Restaurant
7th Most Common Venue     Gym / Fitness Center
8th Most Common Venue               Donut Shop
9th Most Common Venue                      Spa
10th Most Common Venue         Bubble Tea Shop
Name: 47, dtype: object

## Cluster the Queens Borough using K-Means

In [107]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 3, 1, 4, 1, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4], dtype=int32)

In [108]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,4,Middle Eastern Restaurant,Bar,Seafood Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Bakery,Mediterranean Restaurant,Food Truck,Café
1,Queens,Woodside,40.746349,-73.901842,4,Grocery Store,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Bakery,Pizza Place,Pub,Bar,Donut Shop,American Restaurant
2,Queens,Jackson Heights,40.751981,-73.882821,4,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Shoe Store
3,Queens,Elmhurst,40.744049,-73.881656,4,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,South American Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Chinese Restaurant,Gym / Fitness Center,Snack Place,Pizza Place
4,Queens,Howard Beach,40.654225,-73.838138,4,Pharmacy,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Concert Hall,Shipping Store,Sushi Restaurant,Supermarket,Spa,Breakfast Spot


In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Queens Cluster Neighborhood

In [110]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]
queens_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Forest Hills,0,Gym / Fitness Center,Gym,Yoga Studio,Thai Restaurant,Park,Pharmacy,Pizza Place,Convenience Store,Martial Arts School,Italian Restaurant


In [111]:
queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]
queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Maspeth,1,Pizza Place,Diner,Mobile Phone Shop,Chinese Restaurant,Grocery Store,Bank,Deli / Bodega,Bakery,Sports Bar,Sandwich Place
15,Glendale,1,Deli / Bodega,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Brewery,Food Stand,Food Court,Food Truck,Food,Eye Doctor
78,Hammels,1,Beach,Fried Chicken Joint,Dog Run,Food Truck,Fast Food Restaurant,Shoe Store,Gym / Fitness Center,Bus Stop,Bus Station,Diner


In [112]:
queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,2,Deli / Bodega,Automotive Shop,Playground,Pizza Place,Rental Car Location,Donut Shop,South American Restaurant,Spa,Grocery Store,Basketball Court


In [113]:
queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Elmhurst,3,Donut Shop,Gas Station,Bakery,Indie Movie Theater,Chinese Restaurant,American Restaurant,Coffee Shop,Hotel Bar,Ice Cream Shop,Rental Car Location
31,Jamaica Center,3,Pizza Place,Mobile Phone Shop,Clothing Store,Department Store,Performing Arts Venue,Coffee Shop,Sandwich Place,Caribbean Restaurant,Mexican Restaurant,American Restaurant
52,Floral Park,3,Grocery Store,Dosa Place,Indian Restaurant,Basketball Court,Pizza Place,Hobby Shop,Salon / Barbershop,Event Space,Eye Doctor,Falafel Restaurant


In [114]:
queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,4,Middle Eastern Restaurant,Bar,Seafood Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Bakery,Mediterranean Restaurant,Food Truck,Café
1,Woodside,4,Grocery Store,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Bakery,Pizza Place,Pub,Bar,Donut Shop,American Restaurant
2,Jackson Heights,4,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Shoe Store
3,Elmhurst,4,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,South American Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Chinese Restaurant,Gym / Fitness Center,Snack Place,Pizza Place
4,Howard Beach,4,Pharmacy,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Concert Hall,Shipping Store,Sushi Restaurant,Supermarket,Spa,Breakfast Spot
5,Corona,4,Mexican Restaurant,Supermarket,Deli / Bodega,Donut Shop,Chinese Restaurant,Empanada Restaurant,Sandwich Place,South American Restaurant,Restaurant,Bakery
7,Kew Gardens,4,Chinese Restaurant,Cosmetics Shop,Bar,Bank,Indian Restaurant,Donut Shop,Park,Pizza Place,Pharmacy,Deli / Bodega
8,Richmond Hill,4,Latin American Restaurant,Pizza Place,Bank,Lounge,Donut Shop,Metro Station,Caribbean Restaurant,Sandwich Place,Gym / Fitness Center,Clothing Store
9,Flushing,4,Hotpot Restaurant,Korean Restaurant,Chinese Restaurant,Bubble Tea Shop,Construction & Landscaping,Karaoke Bar,Bakery,Asian Restaurant,Café,Gym / Fitness Center
10,Long Island City,4,Coffee Shop,Hotel,Pizza Place,Bar,Café,Mexican Restaurant,Gym / Fitness Center,Donut Shop,Spa,Bubble Tea Shop
